# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 11 2023 8:30AM,254690,10,Enova-11183,Emerginnova,Released
1,Jan 11 2023 8:30AM,254690,10,Enova-11184,Emerginnova,Released
2,Jan 11 2023 8:30AM,254690,10,Enova-11185,Emerginnova,Released
3,Jan 10 2023 3:16PM,254689,19,TR-RA-01112023,"GCH Granules USA, Inc.",Released
4,Jan 10 2023 3:15PM,254688,16,9014977,Sartorius Bioprocess Solutions,Released
5,Jan 10 2023 2:46PM,254686,10,Yusen-9014999,Yusen – 3D Matrix,Executing
6,Jan 10 2023 2:46PM,254685,10,9014401,"Citieffe, Inc.",Executing
7,Jan 10 2023 2:38PM,254684,10,Yusen-9014996,Yusen – 3D Matrix,Executing
8,Jan 10 2023 2:31PM,254682,10,MSP217407,"Methapharm, Inc.",Released
9,Jan 10 2023 2:31PM,254682,10,MSP217399,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,254685,Executing,1
32,254686,Executing,1
33,254688,Released,1
34,254689,Released,1
35,254690,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254685,NaN,1.0,NaN
254686,NaN,1.0,NaN
254688,NaN,NaN,1.0
254689,NaN,NaN,1.0
254690,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254616,0.0,0.0,1.0
254623,7.0,21.0,14.0
254646,0.0,2.0,5.0
254649,0.0,14.0,2.0
254650,0.0,0.0,18.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254616,0,0,1
254623,7,21,14
254646,0,2,5
254649,0,14,2
254650,0,0,18


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254616,0,0,1
1,254623,7,21,14
2,254646,0,2,5
3,254649,0,14,2
4,254650,0,0,18


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254616,,,1
1,254623,7,21,14
2,254646,,2,5
3,254649,,14,2
4,254650,,,18


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 11 2023 8:30AM,254690,10,Emerginnova
3,Jan 10 2023 3:16PM,254689,19,"GCH Granules USA, Inc."
4,Jan 10 2023 3:15PM,254688,16,Sartorius Bioprocess Solutions
5,Jan 10 2023 2:46PM,254686,10,Yusen – 3D Matrix
6,Jan 10 2023 2:46PM,254685,10,"Citieffe, Inc."
7,Jan 10 2023 2:38PM,254684,10,Yusen – 3D Matrix
8,Jan 10 2023 2:31PM,254682,10,"Methapharm, Inc."
10,Jan 10 2023 2:31PM,254681,10,ISDIN Corporation
12,Jan 10 2023 2:29PM,254680,10,Emerginnova
13,Jan 10 2023 1:38PM,254678,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 11 2023 8:30AM,254690,10,Emerginnova,,,3
1,Jan 10 2023 3:16PM,254689,19,"GCH Granules USA, Inc.",,,1
2,Jan 10 2023 3:15PM,254688,16,Sartorius Bioprocess Solutions,,,1
3,Jan 10 2023 2:46PM,254686,10,Yusen – 3D Matrix,,1,
4,Jan 10 2023 2:46PM,254685,10,"Citieffe, Inc.",,1,
5,Jan 10 2023 2:38PM,254684,10,Yusen – 3D Matrix,,1,
6,Jan 10 2023 2:31PM,254682,10,"Methapharm, Inc.",,,2
7,Jan 10 2023 2:31PM,254681,10,ISDIN Corporation,,1,1
8,Jan 10 2023 2:29PM,254680,10,Emerginnova,,1,
9,Jan 10 2023 1:38PM,254678,10,Eywa Pharma Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 8:30AM,254690,10,Emerginnova,3,,
1,Jan 10 2023 3:16PM,254689,19,"GCH Granules USA, Inc.",1,,
2,Jan 10 2023 3:15PM,254688,16,Sartorius Bioprocess Solutions,1,,
3,Jan 10 2023 2:46PM,254686,10,Yusen – 3D Matrix,,1,
4,Jan 10 2023 2:46PM,254685,10,"Citieffe, Inc.",,1,
5,Jan 10 2023 2:38PM,254684,10,Yusen – 3D Matrix,,1,
6,Jan 10 2023 2:31PM,254682,10,"Methapharm, Inc.",2,,
7,Jan 10 2023 2:31PM,254681,10,ISDIN Corporation,1,1,
8,Jan 10 2023 2:29PM,254680,10,Emerginnova,,1,
9,Jan 10 2023 1:38PM,254678,10,Eywa Pharma Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 8:30AM,254690,10,Emerginnova,3,,
1,Jan 10 2023 3:16PM,254689,19,"GCH Granules USA, Inc.",1,,
2,Jan 10 2023 3:15PM,254688,16,Sartorius Bioprocess Solutions,1,,
3,Jan 10 2023 2:46PM,254686,10,Yusen – 3D Matrix,,1,
4,Jan 10 2023 2:46PM,254685,10,"Citieffe, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 8:30AM,254690,10,Emerginnova,3.0,NaN,NaN
1,Jan 10 2023 3:16PM,254689,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
2,Jan 10 2023 3:15PM,254688,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
3,Jan 10 2023 2:46PM,254686,10,Yusen – 3D Matrix,NaN,1.0,NaN
4,Jan 10 2023 2:46PM,254685,10,"Citieffe, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 8:30AM,254690,10,Emerginnova,3.0,0.0,0.0
1,Jan 10 2023 3:16PM,254689,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
2,Jan 10 2023 3:15PM,254688,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
3,Jan 10 2023 2:46PM,254686,10,Yusen – 3D Matrix,0.0,1.0,0.0
4,Jan 10 2023 2:46PM,254685,10,"Citieffe, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5093433,37.0,19.0,0.0
15,1018571,27.0,19.0,0.0
16,254688,1.0,0.0,0.0
19,509312,15.0,21.0,7.0
21,509312,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5093433,37.0,19.0,0.0
1,15,1018571,27.0,19.0,0.0
2,16,254688,1.0,0.0,0.0
3,19,509312,15.0,21.0,7.0
4,21,509312,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,37.0,19.0,0.0
1,15,27.0,19.0,0.0
2,16,1.0,0.0,0.0
3,19,15.0,21.0,7.0
4,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,37.0
1,15,Released,27.0
2,16,Released,1.0
3,19,Released,15.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,0.0,0.0,0.0,7.0,0.0
Executing,19.0,19.0,0.0,21.0,0.0
Released,37.0,27.0,1.0,15.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,0.0,0.0,0.0,7.0,0.0
1,Executing,19.0,19.0,0.0,21.0,0.0
2,Released,37.0,27.0,1.0,15.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,0.0,0.0,0.0,7.0,0.0
1,Executing,19.0,19.0,0.0,21.0,0.0
2,Released,37.0,27.0,1.0,15.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()